In [63]:
from mlflow.tracking import MlflowClient

In [64]:
import mlflow

mlflow.__version__

'3.1.4'

In [65]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

#client side is the usage of mlflow ui using code
client=MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [66]:
#searching for the experiments
client.search_experiments()

[<Experiment: artifact_location='/workspaces/MLops-cookin-/02-MLflow/mlruns/1', creation_time=1758188410351, experiment_id='1', last_update_time=1758188410351, lifecycle_stage='active', name='Mlflow_cooking', tags={}>]

In [67]:
from mlflow.entities import ViewType

# import the top 5 model having less rmse
runs=client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse > 6.6",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [68]:
for run in runs:
    print(f"run id:{run.info.run_id}, rmse:{run.data.metrics['rmse']:.4f}")

run id:10c41ca1c5044913aeb3b399b55ec60a, rmse:6.6196
run id:5b767026cb504cdea56495e21410848d, rmse:6.6279
run id:557bde03037843a2a894c6de53dc819e, rmse:6.6297
run id:76bfe7159b6c4cf586309654e424f5e2, rmse:6.6299
run id:8afca3a497d347bbbbc7d3b03588f47d, rmse:6.6368


In [69]:
import mlflow

In [70]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [71]:
#appending the experiment into model
run_id="m-f664fd8e209f4c739b361698cd2106c7"
model_uri=f"file:///workspaces/MLops-cookin-/02-MLflow/mlruns/1/models/{run_id}/artifacts"
mlflow.register_model(model_uri=model_uri, name="nyc_taxi")

2025/09/21 11:37:19 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/21 11:37:19 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'nyc_taxi' already exists. Creating a new version of this model...
Created version '4' of model 'nyc_taxi'.


<ModelVersion: aliases=[], creation_timestamp=1758454639278, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1758454639278, metrics=None, model_id=None, name='nyc_taxi', params=None, run_id=None, run_link=None, source='file:///workspaces/MLops-cookin-/02-MLflow/mlruns/1/models/m-f664fd8e209f4c739b361698cd2106c7/artifacts', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [72]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1758368041023, deployment_job_id=None, deployment_job_state=None, description='this is nyc taxi predictor\n', last_updated_timestamp=1758454639278, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1758369998896, current_stage='Staging', deployment_job_state=None, description='The model version 3 was transitioned to Staging on 2025-09-20', last_updated_timestamp=1758371253908, metrics=None, model_id=None, name='nyc_taxi', params=None, run_id=None, run_link=None, source='file:///workspaces/MLops-cookin-/02-MLflow/mlruns/1/models/m-f664fd8e209f4c739b361698cd2106c7/artifacts', status='READY', status_message=None, tags={}, user_id=None, version=3>,
  <ModelVersion: aliases=[], creation_timestamp=1758454639278, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1758454639278, metrics=None, model_id=None, name='nyc_taxi', params=None, run_id=None, run_link=None, source='file:///workspaces/ML

In [73]:
model_name="nyc_taxi"
client.get_latest_versions(name=model_name)

/tmp/ipykernel_10574/2720639949.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.get_latest_versions(name=model_name)


[<ModelVersion: aliases=[], creation_timestamp=1758369998896, current_stage='Staging', deployment_job_state=None, description='The model version 3 was transitioned to Staging on 2025-09-20', last_updated_timestamp=1758371253908, metrics=None, model_id=None, name='nyc_taxi', params=None, run_id=None, run_link=None, source='file:///workspaces/MLops-cookin-/02-MLflow/mlruns/1/models/m-f664fd8e209f4c739b361698cd2106c7/artifacts', status='READY', status_message=None, tags={}, user_id=None, version=3>,
 <ModelVersion: aliases=[], creation_timestamp=1758454639278, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1758454639278, metrics=None, model_id=None, name='nyc_taxi', params=None, run_id=None, run_link=None, source='file:///workspaces/MLops-cookin-/02-MLflow/mlruns/1/models/m-f664fd8e209f4c739b361698cd2106c7/artifacts', status='READY', status_message=None, tags={}, user_id=None, version=4>]

In [74]:
model_name="nyc_taxi"
lastest_version=client.get_latest_versions(name=model_name)

for version in lastest_version:
    print(f"version: {version.version}, stage:{version.current_stage}")

version: 3, stage:Staging
version: 4, stage:None


/tmp/ipykernel_10574/1885879352.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  lastest_version=client.get_latest_versions(name=model_name)


In [75]:
#changing the stage version
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Staging",
    archive_existing_versions=False
)

/tmp/ipykernel_10574/2703597925.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1758369998896, current_stage='Staging', deployment_job_state=None, description='The model version 3 was transitioned to Staging on 2025-09-20', last_updated_timestamp=1758454639347, metrics=None, model_id=None, name='nyc_taxi', params=None, run_id=None, run_link=None, source='file:///workspaces/MLops-cookin-/02-MLflow/mlruns/1/models/m-f664fd8e209f4c739b361698cd2106c7/artifacts', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [83]:
model_version=5
stage="Staging"
from datetime import datetime

date=datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version
)

MlflowException: Attempting to update model version with no new field values.

In [84]:
from sklearn.metrics import mean_squared_error

In [85]:
import pandas as pd

In [86]:
def read_dataframe(filename):
    df=pd.read_parquet(filename)
    
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td :td.total_seconds() / 60)
    
    df= df[(df.duration >= 1.0) & (df.duration <=63)]
    
    categorical= ['PULocationID', 'DOLocationID']
    df[categorical]=df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df["PU_DO"] = df["PULocationID"] + '_' + df["DOLocationID"]
    categorical=["PU_DO"]
    numerical=["trip_distance"]
    train_dicts=df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred=model.predict(X_test)
    return {"rmse":mean_squared_error(y_test, y_pred, squared=False)}


In [87]:
df=read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")

In [88]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2021-03-01 00:05:42,2021-03-01 00:14:03,N,1.0,83,129,1.0,1.56,7.50,...,0.5,0.00,0.0,None,0.3,8.80,1.0,1.0,0.0,8.350000
1,2,2021-03-01 00:21:03,2021-03-01 00:26:17,N,1.0,243,235,1.0,0.96,6.00,...,0.5,0.00,0.0,None,0.3,7.30,2.0,1.0,0.0,5.233333
2,2,2021-03-01 00:02:06,2021-03-01 00:22:26,N,1.0,75,242,1.0,9.93,28.00,...,0.5,2.00,0.0,None,0.3,31.30,1.0,1.0,0.0,20.333333
3,2,2021-03-01 00:24:03,2021-03-01 00:31:43,N,1.0,242,208,1.0,2.57,9.50,...,0.5,0.00,0.0,None,0.3,10.80,2.0,1.0,0.0,7.666667
4,1,2021-03-01 00:11:10,2021-03-01 00:14:46,N,1.0,41,151,1.0,0.80,5.00,...,0.5,1.85,0.0,None,0.3,8.15,1.0,1.0,0.0,3.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83822,2,2021-03-31 22:07:00,2021-03-31 22:13:00,None,NaN,41,75,NaN,1.48,8.46,...,0.0,1.44,0.0,None,0.3,10.20,NaN,NaN,NaN,6.000000
83823,2,2021-03-31 22:56:00,2021-03-31 23:13:00,None,NaN,95,95,NaN,0.09,54.25,...,0.0,0.00,0.0,None,0.3,57.30,NaN,NaN,NaN,17.000000
83824,2,2021-03-31 22:36:00,2021-03-31 22:45:00,None,NaN,95,95,NaN,0.66,8.11,...,0.0,0.00,0.0,None,0.3,8.41,NaN,NaN,NaN,9.000000
83825,2,2021-03-31 23:35:00,2021-04-01 00:00:00,None,NaN,37,14,NaN,9.58,36.83,...,0.0,0.00,0.0,None,0.3,39.88,NaN,NaN,NaN,25.000000


In [89]:
path="/workspaces/MLops-cookin-/02-MLflow/mlruns/1/84b53fb51d0f4fc183dd524aefad1b64/artifacts/code/models/preprocessor.b"

In [90]:
client.download_artifacts(run_id="9ec3ca2bef00476088c314f2ab4eb9ec",path='preprocessor',dst_path='.')

'/workspaces/MLops-cookin-/02-MLflow/preprocessor'

In [91]:
import pickle 
with open("preprocessor/preprocessor.b", 'rb') as f_in:
    dv= pickle.load(f_in)

In [92]:
X_test=preprocess(df, dv)

In [93]:
target="duration"
y_test=df[target].values

In [94]:
model_name = "nyc_taxi"
model_uri="/workspaces/MLops-cookin-/02-MLflow/mlruns/1/84b53fb51d0f4fc183dd524aefad1b64/artifacts/model.xgb"

In [95]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 431 ms, sys: 3.4 ms, total: 434 ms
Wall time: 234 ms


{'rmse': 8.653367367321271}

In [97]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_10574/3610659809.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1758454639278, current_stage='Production', deployment_job_state=None, description=None, last_updated_timestamp=1758454806640, metrics=None, model_id=None, name='nyc_taxi', params=None, run_id=None, run_link=None, source='file:///workspaces/MLops-cookin-/02-MLflow/mlruns/1/models/m-f664fd8e209f4c739b361698cd2106c7/artifacts', status='READY', status_message=None, tags={}, user_id=None, version=4>